<a href="https://colab.research.google.com/github/resquilleur/TestWorkNeowoxCC/blob/master/DataPrepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Загрузка и обработка наборов данных

In [ ]:
# загрузим набор данных
path='/content/drive/MyDrive/DataSets/TestWork/'
data_train = pd.read_csv(path+'for_test_train.csv').drop(columns=['Unnamed: 0'])
# посмотрим кол-во экземпляров по классам
data_train.groupby('label').count()

,message
label,
tag_do_not_call,85
tag_no,122
tag_no_now,1
tag_not_now,67
tag_what_company,36
tag_who_is,50
tag_yes,191


In [ ]:
# видим, что у нас есть опечатка в теге
# сравнив подходящий по смыслу тег мы точно понимаем, что это опечатка в название тега 
data_train[data_train['label'] == 'tag_no_now']

,message,label
543,"ты шутишь, у меня нет времени!",tag_no_now


In [ ]:
# получим индекс нужного тега и переименуем его в правильный тег
idx = (data_train['label'] == 'tag_no_now')
data_train.loc[idx, 'label'] = 'tag_not_now'
data_train.groupby('label').count()

,message
label,
tag_do_not_call,85
tag_no,122
tag_not_now,68
tag_what_company,36
tag_who_is,50
tag_yes,191


In [ ]:
# видим дисбаланс классов и в целом датасет небольшой, сделаем аугментацию для повышения качество распознования классов
aug_data_train = pd.DataFrame() # создадим пустой датафрейм
labels = data_train['label'].unique() # список уникальных меток классов

# сделаем так, чтобы обязательно попали все примеры, и добавим сгенерированные бутстрапом, так чтобы в сумме оказалось 200 шт каждого класса
for label in labels:
    n_samples = 0
    aug_data_train = pd.concat([aug_data_train, data_train[data_train['label'] == label]], ignore_index=True)
    if label not in ['tag_yes', 'tag_no']:
        n_samples = 200 - data_train[data_train['label'] == label].count()[0]
    aug_data_train = pd.concat([aug_data_train, data_train[data_train['label'] == label].sample(n=n_samples, replace=True, random_state=42)], ignore_index=True)

aug_data_train

,message,label
0,будьте здоровы!,tag_no
1,не имеется,tag_no
2,ни фига,tag_no
3,вы бредите,tag_no
4,"нет, нашел дурака",tag_no
...,...,...
1108,скажите по новой,tag_who_is
1109,"да, алло, я не расслышал",tag_who_is
1110,"да, алло, я не расслышал",tag_who_is
1111,это кто?,tag_who_is


In [ ]:
data_valid = pd.read_csv(path+'for_test_valid.csv', sep=';') # обязательно использовать такой разделитель, иначе не загрузится.
data_valid # проверочный набор

,Unnamed: 0,message
0,106,зачем это надо
1,162,ну
2,602,какая организация?
3,32,во имя чего?
4,642,вы тут?
5,419,да ну вас!
6,854,"приветствую, а что за организация?"
7,59,да хорошо
8,219,с какой радости?
9,112,и что дальше?


In [ ]:
x_train = aug_data_train['message'].values
y_train = aug_data_train['label'].values

# сделаем one_hot_encoding для набора классов
ohe_ytrain = OneHotEncoder()
y_train = ohe_ytrain.fit_transform(y_train.reshape(-1,1)).toarray()
y_train[0]

array([0., 1., 0., 0., 0., 0.])

In [ ]:
# x_train_normal = []

# for frase in x_train:
#     new_frase = ''
#     words = frase.split()
#     for word in words:
#         word = word.replace('!', '').replace(',', '').replace('?', '')
#         new_frase += f' {ma.normal_forms(word)[0]}'
#     x_train_normal.append(new_frase)   

In [ ]:
# x_train_normal

In [ ]:
# токенизируем слова
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train) # можно ограничится одним токенайзером на аугментированном наборе, так как мы гарантированно добавили все данные в набор

vocabulary = tokenizer.word_index

In [ ]:
x_train_not_aug = data_train['message'].values
y_train_not_aug = data_train['label'].values

y_train_not_aug = ohe_ytrain.fit_transform(y_train_not_aug.reshape(-1,1)).toarray()

x_train_seq_not_aug = tokenizer.texts_to_sequences(x_train_not_aug)
x_train_pad_seq_not_aug = pad_sequences(x_train_seq_not_aug, padding='pre')

In [ ]:
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_train_pad_seq = pad_sequences(x_train_seq, padding='pre')
print(x_train_pad_seq[0])
print(x_train[0])

[  0   0   0   0   0   0   0 322 323]
будьте здоровы!


In [ ]:
x_train_pad_seq.shape

(1113, 9)

In [ ]:
y_train.shape

(1113, 6)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_train_pad_seq, y_train, test_size=0.2, stratify=y_train)